In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import nltk
nltk.download('punkt')


import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\menge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS
        
    def addSentence(self, sentence):
        
        regex = "[\u0618-\u061A|\u064B-\u0653]"

        tokenized_text = megasplit('', sentence)
        
        #print (sentence)
        chars = []
        
        for i in range(len(tokenized_text)):
            if (tokenized_text[i] == ''):
                continue
            if (re.search(regex, tokenized_text[i])):
                continue
                
            if (re.search(regex, tokenized_text[i+1])):
                char = tokenized_text[i] + tokenized_text[i+1]
                chars.append(char)
                self.addWord(char)
                
            else:
                self.addWord(tokenized_text[i])
                                
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
import codecs
import csv
import random
from sklearn.model_selection import train_test_split

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")
    

    FILENAME_TRAIN = "char_basedTrain.data"
    FILENAME_TEST = "char_basedTest.data"
    

    ENCODING = 'utf-8'
    train_pairs = []
    with codecs.open(FILENAME_TRAIN, "r", ENCODING) as fp:
      reader = csv.reader(fp)
      for rows in reader:
        train_pairs.append(rows)
        
    test_pairs = []
    with codecs.open(FILENAME_TEST, "r", ENCODING) as fp:
      reader = csv.reader(fp)
      for rows in reader:
        test_pairs.append(rows)
      
        
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)
    
    
    return input_lang, output_lang, train_pairs, test_pairs


        

In [5]:
def megasplit(pattern, string):
    splits = list((m.start(), m.end()) for m in re.finditer(pattern, string))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits] + [len(string)]
    return [string[start:end] for start, end in zip(starts, ends)]


def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, train_pairs, test_pairs = readLangs(lang1, lang2, reverse)
  
    print("Read %s training sentence pairs" % len(train_pairs))
  
    print("Counting words...")
    for pair in train_pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    
    ##!!!Adding new words in test to training words
    
    print("Read %s test sentence pairs" % len(test_pairs))
    print("Counting words...")
    for pair in test_pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    
    return input_lang, output_lang, train_pairs, test_pairs


input_lang, output_lang, train_pairs, test_pairs = prepareData('NoDiac', 'Diac', False)
print(random.choice(train_pairs))
print(random.choice(test_pairs))



Reading lines...
Read 12004 training sentence pairs
Counting words...
Counted words:
NoDiac 39
Diac 257
Read 1700 test sentence pairs
Counting words...
Counted words:
NoDiac 39
Diac 257
['حتى', 'حتى']
['وجعلكم', 'وجعَلَكم']


In [6]:
print([input_lang.word2index])
print([output_lang.word2index])


[{'ك': 2, 'ل': 3, 'م': 4, 'ا': 5, 'ت': 6, 'أ': 7, 'و': 8, 'ع': 9, 'ب': 10, 'ي': 11, 'ر': 12, 'ط': 13, 'ق': 14, 'ة': 15, 'خ': 16, 'ف': 17, 'ج': 18, 'ه': 19, 'ح': 20, 'د': 21, 'ؤ': 22, 'ن': 23, 'س': 24, 'ز': 25, 'ض': 26, 'ش': 27, 'ء': 28, 'ـ': 29, 'ص': 30, 'غ': 31, 'ذ': 32, 'ئ': 33, 'آ': 34, 'ظ': 35, 'ث': 36, 'إ': 37, 'ى': 38}]
[{'كَ': 2, 'لِ': 3, 'مَ': 4, 'ا': 5, 'تٌ': 6, 'أ': 7, 'و': 8, 'تَ': 9, 'عَ': 10, 'بِ': 11, 'ي': 12, 'رٌ': 13, 'مُ': 14, 'طَ': 15, 'قَ': 16, 'ةٌ': 17, 'خِ': 18, 'ل': 19, 'فُ': 20, 'جَ': 21, 'م': 22, 'ه': 23, 'ح': 24, 'دٌ': 25, 'بْ': 26, 'دُ': 27, 'ؤ': 28, 'ن': 29, 'ب': 30, 'ع': 31, 'د': 32, 'قُ': 33, 'س': 34, 'ف': 35, 'أَ': 36, 'مّ': 37, 'بَ': 38, 'هَ': 39, 'طُ': 40, 'حِ': 41, 'نُ': 42, 'كُ': 43, 'زِ': 44, 'لْ': 45, 'تُ': 46, 'لَ': 47, 'مْ': 48, 'رَ': 49, 'ضَ': 50, 'بّ': 51, 'ك': 52, 'يْ': 53, 'ضِ': 54, 'دّ': 55, 'ج': 56, 'ضُ': 57, 'ر': 58, 'دِ': 59, 'شِ': 60, 'ءُ': 61, 'رِ': 62, 'كْ': 63, 'ضّ': 64, 'عُ': 65, 'ـ': 66, 'شْ': 67, 'هِ': 68, 'سُ': 69, 'صَ': 70, 'غّ': 7

In [7]:

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [8]:
MAX_LENGTH = 50

In [9]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
       

In [10]:
def indexesFromSentence(lang, sentence):
    regex = "[\u0618-\u061A|\u064B-\u0653]"
    tokenized_text = megasplit('', sentence)
        
        #print (sentence)
    chars = []
        
    for i in range(len(tokenized_text)):
        if (tokenized_text[i] == ''):
            continue
        if (re.search(regex, tokenized_text[i])):
            continue
                
        if (re.search(regex, tokenized_text[i+1])):
            char = tokenized_text[i] + tokenized_text[i+1]
            chars.append(lang.word2index[char])                
        else:
            chars.append(lang.word2index[tokenized_text[i]])

    return chars

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])

    return (input_tensor, target_tensor)

In [11]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    #use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    use_teacher_forcing = False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            #decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [12]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [13]:
averageLoss = []
accuracies = []

In [14]:
lr_per_iter = []


def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    criterion = nn.NLLLoss()
        
    for i in range(n_iters):
        print(i)
        numExample = 0
        for pairs in train_pairs:
            training_pair = tensorsFromPair(pairs)
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]
            
            loss = train(input_tensor, target_tensor, encoder,decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss
            numExample += 1
            
            if (numExample % print_every == 0):
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                #print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         #iter, iter / n_iters * 100, print_loss_avg))
                print('%s %d %.4f %d' % (timeSince(start, numExample), numExample, print_loss_avg, i))
      
        accuracy = getAccuracy(encoder1, attn_decoder1)
        accuracies.append(accuracy)
        averageLoss.append(print_loss_total / print_every)
        print(('%.4f,%.4f') % ((print_loss_total / print_every),loss))
        print(accuracy)
      

Plotting results
----------------

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.




In [15]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Evaluation





In [16]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
           #decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)            
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                #decoded_words.append('<EOS>')
                break
            else:
                #if (topi.item() not in output_lang.test_index2word):
                  #decoded_words.append("unk")
                #else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [17]:
def getAccuracy(encoder, decoder, n=10):
    
    correct_sentence_pair = 0
    incorrect_sentence_pair = 0
    
    correct_word_pair = 0
    incorrect_word_pair = 0
        
    correct_diac = 0
    incorrect_diac = 0
    
    regex = "[\u0618-\u061A|\u064B-\u0653]"
 
    count = 0
    sentenceInput = []
    sentenceTarget = []
    sentenceOutput = []
    
    for pair in test_pairs:
        
        output_chars, attentions = evaluate(encoder, decoder, pair[0])
        output_word = ''.join(output_chars)
            
        if(count < 4):
            sentenceInput.append(pair[0])
            sentenceTarget.append(pair[1])
            sentenceOutput.append(output_word)
            count += 1
        else:
            #Accuracy on sentence basis
            
            if (sentenceTarget == sentenceOutput):
                correct_sentence_pair += 1
                #print('correct: ',sentenceTarget, sentenceOutput)


            else:
                incorrect_sentence_pair += 1
                #print('incorrect: ',sentenceTarget, sentenceOutput)

            sentenceInput = []
            sentenceTarget = []
            sentenceOutput = [] 
            
            sentenceInput.append(pair[0])
            sentenceTarget.append(pair[1])
            sentenceOutput.append(output_word)
            count = 1            
            
        
        #Accuracy on word basis
        if (pair[1] == output_word):
            if (re.search(regex, pair[1])):
                correct_diac += 1
                correct_word_pair += 1
        else:
            if (re.search(regex, pair[1])):
                incorrect_diac += 1
                incorrect_word_pair += 1     
  
    return((correct_word_pair/(correct_word_pair+incorrect_word_pair), correct_sentence_pair/(correct_sentence_pair+incorrect_sentence_pair)))

In [18]:
def evaluateRandomly(encoder, decoder, n=10):
    
    correct_sentence_pair = 0
    incorrect_sentence_pair = 0
    
    correct_word_pair = 0
    incorrect_word_pair = 0
        
    correct_diac = 0
    incorrect_diac = 0
    
    regex = "[\u0618-\u061A|\u064B-\u0653]"
 
    count = 0
    sentenceInput = []
    sentenceTarget = []
    sentenceOutput = []
    
    for pair in test_pairs:
        
        output_chars, attentions = evaluate(encoder, decoder, pair[0])
        output_word = ''.join(output_chars)
            
        if(count < 4):
            sentenceInput.append(pair[0])
            sentenceTarget.append(pair[1])
            sentenceOutput.append(output_word)
            count += 1
        else:
            #Accuracy on sentence basis
            
            if (sentenceTarget == sentenceOutput):
                correct_sentence_pair += 1
                print('correct: ',sentenceTarget, sentenceOutput)


            else:
                incorrect_sentence_pair += 1
                print('incorrect: ',sentenceTarget, sentenceOutput)

            sentenceInput = []
            sentenceTarget = []
            sentenceOutput = [] 
            
            sentenceInput.append(pair[0])
            sentenceTarget.append(pair[1])
            sentenceOutput.append(output_word)
            count = 1            
            
        
        #Accuracy on word basis
        if (pair[1] == output_word):
            if (re.search(regex, pair[1])):
                correct_diac += 1
                correct_word_pair += 1
        else:
            if (re.search(regex, pair[1])):
                incorrect_diac += 1
                incorrect_word_pair += 1     
  
    print("accuracy of words:",correct_word_pair/(correct_word_pair+incorrect_word_pair),correct_word_pair,incorrect_word_pair) 
    print("percentage correct diacritic:",correct_diac/correct_word_pair)
    print("Number of all words: ",correct_word_pair+correct_word_pair)
    
    print("accuracy of sentence:",correct_sentence_pair/(correct_sentence_pair+incorrect_sentence_pair),correct_sentence_pair,incorrect_sentence_pair) 
    print("Number of all sentences: ", correct_sentence_pair+incorrect_sentence_pair)

In [19]:
print("input:", input_lang.n_words)
print("train size", len(train_pairs))

input: 39
train size 12004


In [20]:
hidden_size = 256

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)


In [22]:
trainIters(encoder1, attn_decoder1, 100, print_every=1000, learning_rate=0.001)

0
0m 28s (- -1m 31s) 1000 2.8467 0
0m 58s (- -1m 1s) 2000 2.8102 0
1m 26s (- -2m 33s) 3000 2.5937 0
1m 55s (- -2m 4s) 4000 2.6929 0
2m 26s (- -3m 33s) 5000 2.6000 0
2m 57s (- -3m 2s) 6000 2.6935 0
3m 28s (- -4m 31s) 7000 2.6358 0
4m 0s (- -5m 59s) 8000 2.5729 0
4m 32s (- -5m 27s) 9000 2.4818 0
5m 3s (- -6m 56s) 10000 2.4962 0
5m 34s (- -6m 25s) 11000 2.4627 0
6m 6s (- -7m 53s) 12000 2.4160 0
0.0090,2.9223
(0.06156048675733715, 0.0)
1
6m 48s (- -7m 11s) 1000 2.5904 1
7m 19s (- -8m 40s) 2000 2.3983 1
7m 49s (- -8m 10s) 3000 2.1749 1
8m 21s (- -9m 39s) 4000 2.2447 1
8m 52s (- -9m 7s) 5000 2.0868 1
9m 24s (- -10m 35s) 6000 2.2197 1
9m 55s (- -10m 4s) 7000 2.1078 1
10m 29s (- -11m 30s) 8000 2.0711 1
11m 3s (- -12m 56s) 9000 1.9600 1
11m 34s (- -12m 25s) 10000 2.0026 1
12m 8s (- -13m 51s) 11000 1.9673 1
12m 40s (- -13m 20s) 12000 1.9437 1
0.0070,2.1054
(0.08088761632068718, 0.0)
2
13m 32s (- -14m 28s) 1000 2.1318 2
14m 10s (- -15m 50s) 2000 1.9522 2
14m 45s (- -15m 15s) 3000 1.7586 2
15m 20s

0.0009,0.4791
(0.3364352183249821, 0.05188679245283019)
33
204m 10s (- -204m 2s) 1000 0.3895 33
204m 37s (- -205m 28s) 2000 0.3378 33
205m 3s (- -205m 0s) 3000 0.2936 33
205m 30s (- -206m 33s) 4000 0.3449 33
205m 56s (- -206m 5s) 5000 0.3137 33
206m 24s (- -207m 37s) 6000 0.3411 33
206m 53s (- -207m 7s) 7000 0.3200 33
207m 21s (- -208m 40s) 8000 0.3218 33
207m 49s (- -208m 12s) 9000 0.2916 33
208m 15s (- -209m 45s) 10000 0.3335 33
208m 42s (- -209m 18s) 11000 0.3107 33
209m 9s (- -210m 51s) 12000 0.3065 33
0.0009,0.4968
(0.3299928418038654, 0.04481132075471698)
34
209m 46s (- -210m 26s) 1000 0.3595 34
210m 13s (- -211m 52s) 2000 0.3173 34
210m 39s (- -211m 24s) 3000 0.2833 34
211m 6s (- -212m 56s) 4000 0.3435 34
211m 33s (- -212m 29s) 5000 0.3097 34
212m 4s (- -213m 58s) 6000 0.3371 34
212m 30s (- -213m 31s) 7000 0.3121 34
212m 58s (- -213m 3s) 8000 0.3097 34
213m 25s (- -214m 36s) 9000 0.2872 34
213m 51s (- -214m 9s) 10000 0.3302 34
214m 18s (- -215m 43s) 11000 0.3044 34
214m 44s (- -

381m 47s (- -382m 14s) 10000 0.1816 64
382m 17s (- -383m 44s) 11000 0.1590 64
382m 44s (- -383m 17s) 12000 0.1717 64
0.0008,0.1677
(0.3750894774516822, 0.04716981132075472)
65
383m 21s (- -383m 1s) 1000 0.2021 65
383m 48s (- -384m 23s) 2000 0.1775 65
384m 15s (- -385m 52s) 3000 0.1585 65
384m 41s (- -385m 23s) 4000 0.1888 65
385m 8s (- -386m 55s) 5000 0.1767 65
385m 36s (- -386m 27s) 6000 0.1968 65
386m 2s (- -386m 0s) 7000 0.1746 65
386m 30s (- -387m 32s) 8000 0.1618 65
386m 58s (- -387m 3s) 9000 0.1633 65
387m 26s (- -388m 35s) 10000 0.1788 65
387m 54s (- -388m 7s) 11000 0.1547 65
388m 21s (- -389m 40s) 12000 0.1704 65
0.0006,0.4345
(0.4037222619899785, 0.08018867924528301)
66
388m 58s (- -389m 25s) 1000 0.1918 66
389m 25s (- -390m 46s) 2000 0.1717 66
389m 52s (- -390m 15s) 3000 0.1534 66
390m 20s (- -391m 45s) 4000 0.1849 66
390m 46s (- -391m 18s) 5000 0.1788 66
391m 13s (- -392m 50s) 6000 0.1885 66
391m 39s (- -392m 23s) 7000 0.1705 66
392m 7s (- -393m 55s) 8000 0.1565 66
392m 34s 

560m 26s (- -561m 37s) 8000 0.1252 96
560m 53s (- -561m 10s) 9000 0.1257 96
561m 21s (- -562m 41s) 10000 0.1395 96
561m 51s (- -562m 11s) 11000 0.1167 96
562m 17s (- -563m 44s) 12000 0.1255 96
0.0002,0.0519
(0.44595561918396565, 0.09669811320754718)
97
562m 55s (- -563m 38s) 1000 0.1513 97
563m 22s (- -564m 54s) 2000 0.1305 97
563m 48s (- -564m 22s) 3000 0.1217 97
564m 15s (- -565m 52s) 4000 0.1472 97
564m 42s (- -565m 24s) 5000 0.1480 97
565m 9s (- -566m 55s) 6000 0.1514 97
565m 36s (- -566m 28s) 7000 0.1387 97
566m 3s (- -566m 0s) 8000 0.1209 97
566m 33s (- -567m 30s) 9000 0.1226 97
567m 1s (- -567m 1s) 10000 0.1357 97
567m 28s (- -568m 34s) 11000 0.1151 97
567m 55s (- -568m 7s) 12000 0.1209 97
0.0002,0.1170
(0.4330708661417323, 0.09669811320754718)
98
568m 32s (- -568m 1s) 1000 0.1559 98
568m 59s (- -569m 17s) 2000 0.1353 98
569m 26s (- -570m 45s) 3000 0.1223 98
569m 52s (- -570m 15s) 4000 0.1517 98
570m 19s (- -571m 47s) 5000 0.1473 98
570m 47s (- -571m 18s) 6000 0.1555 98
571m 13s

In [23]:
print(accuracies)

[(0.06156048675733715, 0.0), (0.08088761632068718, 0.0), (0.09520400858983537, 0.0), (0.11954187544738726, 0.0023584905660377358), (0.13743736578382248, 0.0047169811320754715), (0.15604867573371511, 0.007075471698113208), (0.16607015032211883, 0.01179245283018868), (0.1732283464566929, 0.01650943396226415), (0.18539727988546886, 0.009433962264150943), (0.19470293486041518, 0.01650943396226415), (0.20687186828919113, 0.01650943396226415), (0.21116678596993557, 0.018867924528301886), (0.22476735862562633, 0.018867924528301886), (0.2304939155332856, 0.018867924528301886), (0.24051539012168932, 0.02122641509433962), (0.2448103078024338, 0.014150943396226415), (0.25841088045812455, 0.018867924528301886), (0.26055833929849676, 0.014150943396226415), (0.26055833929849676, 0.01650943396226415), (0.2691481746599857, 0.02358490566037736), (0.2791696492483894, 0.02830188679245283), (0.2755905511811024, 0.02122641509433962), (0.2877594846098783, 0.018867924528301886), (0.29062276306370793, 0.01650

In [24]:
print(averageLoss)

[0.009008000646318707, 0.006992445610818409, 0.005554405791418893, 0.005029102592241197, 0.0040089975907689045, 0.003503235061963399, 0.0030354350805282594, 0.002870013816016061, 0.002808413803577423, 0.0025034606882504058, 0.002472374564125424, 0.0022908120694614597, 0.00226893660284224, 0.0022059324866249446, 0.0022273826258523127, 0.0019862104029882525, 0.0020043003615878878, 0.0018520967222395398, 0.0018083471400397164, 0.0017214558124542237, 0.0016332379182179768, 0.001570883722532363, 0.001506655840646653, 0.0013120586304437546, 0.0015462374119531539, 0.001375198755945478, 0.0014157600743430001, 0.001280453534353347, 0.0013049840983890353, 0.001245203273636954, 0.0010915612947373164, 0.0010311701865423292, 0.0009068450700669061, 0.0009374752725873674, 0.0008246172325951712, 0.000818612155460176, 0.000669302225112915, 0.0007395216113045102, 0.000836838336217971, 0.0006734102283205305, 0.00063344749382564, 0.0006931530918393815, 0.0007635596479688371, 0.0007878826232183547, 0.00068

In [25]:
evaluateRandomly(encoder1, attn_decoder1)

incorrect:  ['الخطابي', 'أَن', 'حَبَّةَ', 'العُرَنيَّ'] ['الخطابي', 'أَن', 'حَبّةَ', 'العُرَنيّ']
incorrect:  ['أَرض', 'في', 'أَنه', 'مؤَنث'] ['أَرض', 'في', 'أَنه', 'مُؤَنّثٌ']
incorrect:  ['على', 'أَحَرَّ', 'كأَنه', 'أَراد'] ['على', 'أَحَرّ', 'كأَنه', 'أَراد']
incorrect:  ['حَرَّ', 'العبدُ', 'يَحَرُّ', 'حَرارَةً'] ['حَرّ', 'العبدُ', 'يَحَرّ', 'حَرارةً']
incorrect:  ['إِلى', 'أَهله', 'أَي', 'صاحبه'] ['إِلى', 'أَهله', 'أَي', 'صاحبَه']
incorrect:  ['عليها', 'كُلُّ', 'بِكْرٍ', 'حُرَّةٍ'] ['عليها', 'كلّ', 'بِكْرٍ', 'حَرّةٌ']
incorrect:  ['من', 'الحُسن', 'إِلا', 'أَن'] ['من', 'الحُسْنِ', 'إِلا', 'أَن']
incorrect:  ['ويقال', 'ساقُ', 'حُرٍّ', 'صَوْتُ'] ['ويقال', 'ساق', 'حَرّ', 'صَوْتِ']
incorrect:  ['سَاقَ', 'حُرٍّ', 'إِن', 'كان'] ['ساق', 'حَرّ', 'إِنّ', 'كان']
incorrect:  ['عَرَفْتُ', 'من', 'ضَرْب', 'الحَرِيرِ'] ['عَرفْت', 'من', 'ضرْب', 'الحَريرِ']
incorrect:  ['للضأْن', 'وفي', 'المحكَم', 'وحَرِّ'] ['للضأْن', 'وفي', 'المحكم', 'وحَرّ']
incorrect:  ['فقال', 'القَبِيلة', 'إِلا', 'أَنهم'] ['فقال

incorrect:  ['اعْتَمَرْ', 'مَغْزًى', 'بَعِيداً', 'من'] ['اعتَمَرْ', 'مَغْزًى', 'بَعِيداً', 'من']
incorrect:  ['قصد', 'مَغْزًى', 'بعيداًوضبَرَ', 'جَمعَ'] ['قصد', 'مَغْزًى', 'بعيداًوضبَرَ', 'جَمع']
correct:  ['قوائمه', 'ليَثِبَ', 'والعُمْرةُ', 'أَن'] ['قوائمه', 'ليَثِبَ', 'والعُمْرةُ', 'أَن']
correct:  ['يَبْنِيَ', 'الرجلُ', 'بامرأَته', 'في'] ['يَبْنِيَ', 'الرجلُ', 'بامرأَته', 'في']
incorrect:  ['أَهلها', 'فإِن', 'نقلها', 'إِلى'] ['أَهلها', 'فإِنْ', 'نقلها', 'إِلى']
incorrect:  ['رَيْحانٍ', 'الطَّيِّب', 'الثناء', 'الطَّيِّب'] ['رَيْحانٍ', 'الطّيّب', 'الثناء', 'الطّيّب']
incorrect:  ['وهو', 'الآسوالعِمَارة', 'والعَمارة', 'التحيّة'] ['وهو', 'الآسوالعِمَارة', 'والعَمارة', 'التّحيّة']
incorrect:  ['عَمّرَك', 'اللهُ', 'وحيّاك', 'وليس'] ['عَمْرَك', 'اللهُ', 'وحيّاك', 'وليس']
correct:  ['للحُرّة', 'خِمار', 'ولا', 'صَوْقَعة'] ['للحُرّة', 'خِمار', 'ولا', 'صَوْقَعة']
incorrect:  ['قامَتْ', 'تُصَلّي', 'والخِمارُ', 'مِن'] ['قامَتْ', 'تُصَلّي', 'والخِمارُ', 'من']
incorrect:  ['عَمَر', 'ربَّه', 'عبَدَ

incorrect:  ['يُسَمَّى', 'به', 'الحَيُّ', 'وعُمَرُ'] ['يسمى', 'به', 'الحيّ', 'وعُمْرُ']
incorrect:  ['وعُوَيْمِرٌ', 'وعَمَّارٌ', 'ومَعْمَرٌ', 'وعِمْرانُ'] ['وعُويمر', 'وعَمّارٌ', 'ومَعْمَرٌ', 'وعِمْرانُ']
incorrect:  ['وعُمارَةُ', 'ويَعْمَرُ', 'كيَفْعَلُ', 'أسْماءٌ'] ['وعمارةً', 'ويَعْمَرُ', 'كيَفْعَلُ', 'أَسْماءُ']
incorrect:  ['والعَمْرانِ', 'عَمْرُو', 'بنُ', 'جابِرٍ'] ['والعَمَران', 'عمرو', 'بن', 'جابِرِ']
incorrect:  ['وبَدْرُ', 'بنُ', 'عَمْرٍو', 'واللَّحْمَتانِ'] ['وبَدْرُ', 'بن', 'عمرو', 'واللحَمتانِ']
incorrect:  ['على', 'اللَّهاةِ', 'والعَامِرانِ', 'ابنُ'] ['على', 'اللّهاةِ', 'والعامِرانِ', 'ابن']
incorrect:  ['مالِكٍ', 'وابنُ', 'الطُّفَيْلِ', 'والعُمَرانِ'] ['مالُك', 'وابنَ', 'الطّفَيْلِ', 'والعَمَران']
incorrect:  ['عُمَرُ', 'وعُمَرُ', 'بنُ', 'عبدِ'] ['عمر', 'وعَمّر', 'بن', 'عبد']
incorrect:  ['العَزيزِ', 'وعَمْرَوَيْهِ', 'أعْجَمِيٌّ', 'وأبو'] ['العزيز', 'وعَمّرَوَيْهِ', 'أَعجمي', 'وأَبو']
incorrect:  ['عَمْرَةَ', 'كُنْيَةُ', 'الإِفلاسِ', 'والجُوعِ'] ['عَمْرة', 'كُنْيَةُ', 'ا